# Credit Risk Resampling Techniques

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [46]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [183]:
# Load the data
# file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv('LoanStats_2019Q1.csv', skiprows=1)
df = df.loc[:, columns].copy()

df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N
2,5000,17.97%,180.69,MORTGAGE,62000.0,Not Verified,Mar-2019,Issued,n,19.82,...,66.7,50.0,0.0,0.0,255738.0,31615.0,9400.0,39938.0,N,N
3,20000,8.19%,628.49,MORTGAGE,200000.0,Not Verified,Mar-2019,Issued,n,22.66,...,95.7,22.2,0.0,0.0,448069.0,84744.0,49400.0,105180.0,N,N
4,12000,15.57%,289.09,MORTGAGE,49000.0,Source Verified,Mar-2019,Issued,n,13.47,...,86.1,14.3,0.0,0.0,189260.0,106025.0,24400.0,68860.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,30000,12.40%,1002.18,MORTGAGE,125000.0,Not Verified,Mar-2019,Issued,n,13.44,...,96.0,62.5,0.0,0.0,435698.0,41177.0,23900.0,37903.0,N,N
1340,10000,10.33%,324.23,RENT,59000.0,Source Verified,Mar-2019,Issued,n,15.24,...,80.0,0.0,0.0,0.0,21247.0,16448.0,10800.0,7647.0,N,N
1341,20000,8.19%,407.35,MORTGAGE,110000.0,Not Verified,Mar-2019,Issued,n,13.83,...,89.5,0.0,0.0,0.0,351051.0,46962.0,46000.0,38643.0,N,N
1342,30000,15.57%,722.71,RENT,102000.0,Source Verified,Mar-2019,Issued,n,11.62,...,100.0,42.9,0.0,0.0,69950.0,38278.0,47000.0,20050.0,N,N


In [184]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

### filling Null Values
df = df.fillna(method='ffill')

# Drop the null rows
df = df.dropna()

In [185]:
# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
80,35000,13.08%,797.80,MORTGAGE,125890.0,Verified,Mar-2019,Fully Paid,n,30.48,...,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0,N,N
82,20000,22.50%,769.00,RENT,75000.0,Not Verified,Mar-2019,Current,n,24.37,...,100.0,100.0,0.0,0.0,77797.0,55014.0,47000.0,30797.0,N,N
93,10500,17.19%,375.35,RENT,66000.0,Source Verified,Mar-2019,Current,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
99,25000,20.00%,929.09,MORTGAGE,105000.0,Verified,Mar-2019,Current,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
132,20000,20.00%,529.88,MORTGAGE,56000.0,Verified,Mar-2019,Current,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,30000,20.00%,794.82,MORTGAGE,144000.0,Not Verified,Mar-2019,Current,n,11.55,...,100.0,12.5,1.0,0.0,358812.0,46902.0,34000.0,20612.0,N,N
1317,25000,13.90%,853.23,MORTGAGE,100000.0,Not Verified,Mar-2019,Current,n,12.43,...,89.7,50.0,0.0,0.0,176667.0,44638.0,13400.0,28367.0,N,N
1318,8000,6.46%,245.05,MORTGAGE,90000.0,Not Verified,Mar-2019,Current,n,24.60,...,100.0,0.0,0.0,0.0,523124.0,106234.0,37000.0,122928.0,N,N
1326,20000,8.19%,628.49,MORTGAGE,150000.0,Source Verified,Mar-2019,Current,n,15.52,...,86.4,20.0,0.0,0.0,411219.0,102152.0,34000.0,106371.0,N,N


In [186]:
# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,35000,0.1308,797.80,MORTGAGE,125890.0,Verified,Mar-2019,Fully Paid,n,30.48,...,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0,N,N
1,20000,0.2250,769.00,RENT,75000.0,Not Verified,Mar-2019,low_risk,n,24.37,...,100.0,100.0,0.0,0.0,77797.0,55014.0,47000.0,30797.0,N,N
2,10500,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
3,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
4,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N


In [187]:
### Getting Dummies Data
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

# Split the Data into Training and Testing

In [188]:
# Create our features
X = df.drop(columns='loan_status')


# # Create our target
yy = {'loan_status': list(df['loan_status'].values)}
y = pd.DataFrame(yy)

In [189]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.0,74.0,74.000000,74.000000,...,74.000000,74.000000,74.000000,74.0,74.000000,74.000000,74.000000,74.000000,74.0,74.0
mean,17952.027027,0.133351,523.842162,1.081081,82199.864865,0.851351,0.0,0.0,18.860946,0.202703,...,95.497297,36.451351,0.148649,0.0,163726.945946,57068.081081,24067.567568,49874.986486,0.0,0.0
std,9715.226678,0.048460,277.888959,0.975828,41472.824072,0.788312,0.0,0.0,11.126093,0.496004,...,8.167711,33.977770,0.358170,0.0,157915.249140,68925.502912,21796.186094,64588.647664,0.0,0.0
min,2400.000000,0.064600,74.730000,0.000000,16323.000000,0.000000,0.0,0.0,0.920000,0.000000,...,54.500000,0.000000,0.000000,0.0,8500.000000,1848.000000,0.000000,0.000000,0.0,0.0
25%,10000.000000,0.083450,318.377500,0.000000,56375.000000,0.000000,0.0,0.0,11.527500,0.000000,...,93.850000,0.000000,0.000000,0.0,51119.000000,18983.500000,9725.000000,12675.000000,0.0,0.0
50%,15900.000000,0.139000,450.385000,2.000000,73750.000000,1.000000,0.0,0.0,17.905000,0.000000,...,100.000000,33.300000,0.000000,0.0,106527.500000,41158.500000,17650.000000,33202.500000,0.0,0.0
75%,25000.000000,0.164000,698.150000,2.000000,100000.000000,1.000000,0.0,0.0,24.042500,0.000000,...,100.000000,64.300000,0.000000,0.0,219247.000000,65435.250000,31325.000000,65339.500000,0.0,0.0
max,40000.000000,0.288000,1537.990000,2.000000,309600.000000,2.000000,0.0,0.0,62.800000,2.000000,...,100.000000,100.000000,1.000000,0.0,588300.000000,418934.000000,106300.000000,383625.000000,0.0,0.0


In [190]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    58
0    16
Name: loan_status, dtype: int64

In [191]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1,shuffle=True)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [238]:
from imblearn.over_sampling import RandomOverSampler
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
x_train, y_train = ros.fit_resample(X_train.values, Y_train.values)

In [239]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
# Train the Logistic Regression model using the resampled data
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
pred = clf.predict(X_test.values)

In [240]:
# Calculated the balanced accuracy score
print(f'Accuracy Score: {balanced_accuracy_score(pred, Y_test.values.flatten())}')

Accuracy Score: 1.0


In [241]:
# Display the confusion matrix
print('Confusion Matrix')
confusion_matrix(pred, Y_test.values.flatten())

Confusion Matrix


array([[2, 0],
       [0, 6]])

In [242]:
# Print the imbalanced classification report
import imblearn.metrics as imb
print(imb.classification_report_imbalanced(Y_test.values.flatten(), pred,     target_names=['Class_0', 'Class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    Class_0       1.00      1.00      1.00      1.00      1.00      1.00         2
    Class_1       1.00      1.00      1.00      1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8



### SMOTE Oversampling

In [259]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
x_train, y_train = sm.fit_resample(X_train.values, Y_train.values)

In [260]:
# Train the Logistic Regression model using the resampled data
clf = LogisticRegression(random_state=1).fit(x_train, y_train)
pred = clf.predict(X_test.values)

In [261]:
# Calculated the balanced accuracy score
print(f'Accuracy Score: {balanced_accuracy_score(pred, Y_test.values.flatten())}')

Accuracy Score: 1.0


In [262]:
# Display the confusion matrix
print('Confusion Matrix')
confusion_matrix(pred, Y_test.values.flatten())

Confusion Matrix


array([[2, 0],
       [0, 6]])

In [263]:
# Print the imbalanced classification report
print(imb.classification_report_imbalanced(Y_test.values.flatten(), pred,     target_names=['Class_0', 'Class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    Class_0       1.00      1.00      1.00      1.00      1.00      1.00         2
    Class_1       1.00      1.00      1.00      1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [264]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
x_train, y_train = cc.fit_resample(X_train.values, Y_train.values)

In [265]:
# Train the Logistic Regression model using the resampled data
clf = LogisticRegression(random_state=1).fit(x_train, y_train)
pred = clf.predict(X_test.values)

In [266]:
# Calculated the balanced accuracy score
print(f'Accuracy Score: {balanced_accuracy_score(pred, Y_test.values.flatten())}')

Accuracy Score: 1.0


In [267]:
# Display the confusion matrix
print('Confusion Matrix')
confusion_matrix(pred, Y_test.values.flatten())

Confusion Matrix


array([[2, 0],
       [0, 6]])

In [268]:
# Print the imbalanced classification report
print(imb.classification_report_imbalanced(Y_test.values.flatten(), pred,     target_names=['Class_0', 'Class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    Class_0       1.00      1.00      1.00      1.00      1.00      1.00         2
    Class_1       1.00      1.00      1.00      1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [269]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smt = SMOTEENN(random_state=1)
x_train, y_train = smt.fit_resample(X_train.values, Y_train.values)

In [270]:
# Train the Logistic Regression model using the resampled data
clf = LogisticRegression(random_state=1).fit(x_train, y_train)
pred = clf.predict(X_test.values)

In [271]:
# Calculated the balanced accuracy score
print(f'Accuracy Score: {balanced_accuracy_score(pred, Y_test.values.flatten())}')

Accuracy Score: 1.0


In [272]:
# Display the confusion matrix
print('Confusion Matrix')
confusion_matrix(pred, Y_test.values.flatten())

Confusion Matrix


array([[2, 0],
       [0, 6]])

In [274]:
# Print the imbalanced classification report
print(imb.classification_report_imbalanced(Y_test.values.flatten(), pred,     target_names=['Class_0', 'Class_1']))

                   pre       rec       spe        f1       geo       iba       sup

    Class_0       1.00      1.00      1.00      1.00      1.00      1.00         2
    Class_1       1.00      1.00      1.00      1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00      1.00      1.00      1.00         8

